In [5]:
# 한국어 전처리를 위한 konlpy
# !pip install konlpy

In [7]:
# 한국어 전처리를 위한 Twitter 
# !pip install Twitter

In [9]:
# word2vec 을 하기 위한 gensim 모듈 설치
# !pip install gensim

In [46]:
import os
import pymysql
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec

In [47]:
# 본인 컴퓨터의 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [48]:
os.environ["JAVA_HOME"]

'C:\\Program Files\\Java\\jdk-11'

### `Twitter` 예시 사용 확인

##### Twitter 모듈의 이름이 `Okt` 로 바뀜 : "Twitter" has changed to "Okt" since KoNLPy v0.4.5.

In [49]:
from konlpy.tag import Twitter

In [147]:
# 한국어 문자열을 단어로 나누고 품사를 리턴하는 Twitter() 객체 생성
twitter = Twitter()


c:\workspace\venv\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [15]:
# twitter 예시
## 단어와 품사 분리
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워요!!")

[('안녕', 'Noun'),
 ('하십니까', 'Verb'),
 ('좋은', 'Adjective'),
 ('아침', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반가워요', 'Adjective'),
 ('!!', 'Punctuation')]

In [64]:
# stemp=True : 동사인 '하십니까 반가워' 는 기본 동사형 '안녕하다' 와 '반갑다' 로 변환
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워요 집으로 뛰어 갔다 정말 참으로 좋다", stem=True)
## 리스트 안에 튜플 형태로 값 반환.

[('안녕', 'Noun'),
 ('하다', 'Verb'),
 ('좋다', 'Adjective'),
 ('아침', 'Noun'),
 ('이다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반갑다', 'Adjective'),
 ('집', 'Noun'),
 ('으로', 'Josa'),
 ('뛰다', 'Verb'),
 ('가다', 'Verb'),
 ('정말', 'Noun'),
 ('참으로', 'Noun'),
 ('좋다', 'Adjective')]

#### 텍스트를 전처리 하는 함수

In [50]:
# text : 전처리할 영화 줄거리
from konlpy.tag import Twitter # Twitter 모듈의 이름이 Okt로 바뀜 
                            # "Twitter" has changed to "Okt" since KoNLPy v0.4.5.

twitter = Twitter()

def preprocessingText(text):
    # 전처리한 결과를 저장할 리스트
    stems = []
    
    # text를 단어로 나누고 품사를 붙여서 리턴
    # stem=True : 단어의 기본형으로 변환
    tagged_review = twitter.pos(text, stem=True)
    
    # tagged_review 에서 하나를 꺼내서 word, pos에 대입
    # 단어는 word, 품사는 pos에 할당 <- tagged_review 안의 하나 하나의 값이 튜플로 이루어짐.
    for word, pos in tagged_review:
        # len(word) : 단어의 글자 수가 2글자 이상이면
        ## 품사가 Noun(명사) 혹은 Adjective(형용사) 일 경우에만 word를  stems에 추가
        if pos =="Noun" or pos =="Adjective" :
            stems.append(word)
            
    # 리스트 stems 를 문자열로 만들어 (공백 기준으로) (" ".join(stems)) 리턴
    return " ".join(stems)

c:\workspace\venv\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
preprocessingText("아버지가 방에 들어가셨다 나는 더위를  먹었다 수박을 먹었다")

'아버지 방 나 더위 수박'

In [51]:
preprocessingText("대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 뒤, ‘마석도’(마동석)는 새로운 팀원들과 함께 살인사건을 조사한다. 사건 조사 중, ‘마석도’는 신종 마약 사건이 연루되었음을 알게 되고 수사를 확대한다. 한편, 마약 사건의 배후인 '주성철'(이준혁)은 계속해서 판을 키워가고 약을 유통하던 일본 조직과 '리키'(아오키 무네타카)까지 한국에 들어오며 사건의 규모는 점점 더 커져가는데... 나쁜 놈들 잡는 데 이유 없고 제한 없다. 커진 판도 시원하게 싹 쓸어버린다!")

'대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마 석도 마동석 새롭다 팀원 살인 사건 조사 사건 조사 중 마 석도 신종 마약 사건 연루 알 수사 확대 한편 마약 사건 배후 주성철 이준혁 은 계속 판 약 유통 일본 조직 리키 아오키 무 타카 한국 사건 규모 점점 더 나쁘다 놈 데 이유 없다 제한 없다 판도 시원하다 싹'

# DB와 연결 후 테이블 데이터 줄거리 `preprocessingText()` 함수를 통해 전처리 하기 

In [142]:
db = pymysql.connect(
    # host = "aws db 엔드포인트",
    host="",
    port=3306,
    user='admin',
    # passwd='관리자 계정 비밀번호',
    passwd="",
    db='movie_db',
    charset='utf8'
)

In [53]:
sql = "SELECT * FROM movie;"

movie_df = pd.read_sql(sql, db)
# movie_df

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18000\1050188224.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(sql, db)


In [12]:
# 줄거리 데이터를 전처리한 예시
preprocessingText(movie_df["synopsis"][0])

'대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마 석도 마동석 새롭다 팀원 살인 사건 조사 사건 조사 중 마 석도 신종 마약 사건 연루 알 수사 확대 한편 마약 사건 배후 주성철 이준혁 은 계속 판 약 유통 일본 조직 리키 아오키 무 타카 한국 사건 규모 점점 더 나쁘다 놈 데 이유 없다 제한 없다 판도 시원하다 싹'

In [54]:
# 데이터프레임에서 "synopsis_clear" 컬럼을 추가하고
## np.NaN(결측치) 할당
movie_df["synopsis_clear"] = np.NaN
# movie_df

In [55]:
row_num = len(movie_df) # 데이터프레임 행 개수 조회
row_num

670

In [56]:
for i in tqdm(range(row_num)):
    try:
        # 줄거리(synopsis) 컬럼의 데이터
        synopsis = movie_df.loc[i, "synopsis"]
        
        # 해당 줄거리 데이터를 preprocessingText() 함수를 통해
        ## 명사, 형용사 단어의 기본형을 리턴해
        ### synopsis_clear 컬럼에 할당
        movie_df.loc[i,"synopsis_clear"] = preprocessingText(synopsis)
    
    except Exception as e:
        print(e)
        # 에러 발생 시 결측치로 처리
        movie_df.loc[i,"synopsis_clear"] = np.NaN

100%|██████████| 670/670 [00:06<00:00, 102.20it/s]


In [57]:
movie_df["synopsis_clear"].head()

0    대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마...
1    이 근처 폐허 없다 문 있다 규슈 한적하다 마을 살 있다 소녀 스즈메 문 여행 중인...
2    불 물 공기 흙 개 원소 살 있다 엘리 멘트 시티 재치 있다 불 열정 넘치다 앰버 ...
3                        전국 제패 북산 농구부 인방 꿈 열정 도전 그린 영화
4    가모라 를 슬픔 있다 피터 퀼 이 위기 처 은하계 동료 위해 다시 한번 가디언즈 팀...
Name: synopsis_clear, dtype: object

In [58]:
# "synopsis_clear" 컬럼 데이터에 마지막에 " " 공백을 추가해
## 해당 문자열 데이터를 공백기준으로 잘라서 리스트로 변환 후
### "synopsis_clear_list" 컬럼을 추가하고 여기에 저장

movie_df["synopsis_clear"] = movie_df["synopsis_clear"].astype(str) + " "

movie_df["synopsis_clear_list"] = movie_df["synopsis_clear"].apply(lambda x : x.split(" "))

movie_df["synopsis_clear_list"].head()

0    [대체, 불가, 괴물, 형사, 마, 석도, 서울, 광수, 발탁, 베트남, 납치, 살...
1    [이, 근처, 폐허, 없다, 문, 있다, 규슈, 한적하다, 마을, 살, 있다, 소녀...
2    [불, 물, 공기, 흙, 개, 원소, 살, 있다, 엘리, 멘트, 시티, 재치, 있다...
3           [전국, 제패, 북산, 농구부, 인방, 꿈, 열정, 도전, 그린, 영화, ]
4    [가모라, 를, 슬픔, 있다, 피터, 퀼, 이, 위기, 처, 은하계, 동료, 위해,...
Name: synopsis_clear_list, dtype: object

## `Word2Vec` 을 이용해 각 단어들을 학습해 `Vector` 형태로 생성해서 최종 DB에 저장

In [16]:
from gensim.models.word2vec import Word2Vec

In [60]:
word2vec = Word2Vec(movie_df["synopsis_clear_list"], # 리스트 형태의 데이터
                    sg=1,               # 0 : CBOW, 1: Skip-gram
                    vector_size= 80,    # Word2vector 의 열의 수
                    window= 3,          # 고려할 앞뒤 폭 (앞뒤 3단어)
                    min_count= 2,       # 사용할 단어의 최소 빈도 (2회 이하 단어 무시)
                    workers= 4         # 동시에 처리할 작업 수 (코어 수와 비슷하게 설정)
                    )

In [19]:
# Vector 형태로 변환한 단어 리스트 조회

# word2vec.wv.key_to_index
word2vec.wv.key_to_index.keys()

dict_keys(['', '그', '있다', '시작', '수', '위해', '것', '없다', '자신', '사람', '이', '의', '사랑', '날', '은', '그녀', '두', '속', '친구', '를', '세상', '알', '가족', '과', '새롭다', '모든', '같다', '곳', '꿈', '집', '사건', '세계', '그렇다', '전', '살', '엄마', '이야기', '마음', '다시', '서로', '영화', '위', '시간', '남자', '과연', '일', '중', '삶', '앞', '아이', '마을', '후', '뒤', '못', '위기', '다른', '하나', '온', '순간', '딸', '향', '사이', '구', '모두', '아들', '소녀', '아내', '인생', '우리', '가장', '운명', '더', '내', '비밀', '학교', '마지막', '기억', '한편', '말', '지구', '안', '힘', '단', '점점', '사실', '대한', '아버지', '과거', '발견', '세', '둘', '채', '최고', '끝', '하루', '유일하다', '이자', '상황', '정체', '때', '만난', '특별하다', '남편', '생활', '팀', '이름', '여자', '죽음', '마주', '로', '간다', '소년', '처', '작전', '나', '생각', '처음', '준비', '경찰', '현실', '감정', '건', '조직', '인간', '도시', '누구', '결혼', '길', '이제', '아니다', '분', '이후', '밤', '관계', '아빠', '존재', '결심', '지금', '살인', '어떻다', '도착', '인하다', '희망', '모험', '무엇', '일상', '사고', '무대', '도움', '등', '통해', '홀로', '수사', '전쟁', '눈', '문', '모습', '카', '진실', '선택', '새', '뜨겁다', '여행', '마', '명의', '리', '계속', '잠시', '아름답다', '연인', '돈', '동생', 

In [61]:
# 액션의 인덱스 (빈도 수 순위 0이 가장 빈도 수 높은 단어)

word2vec.wv.key_to_index["액션"]

345

In [62]:
# 로맨스의 인덱스 (빈도 수 순위 0이 가장 빈도 수 높은 단어)

word2vec.wv.key_to_index["로맨스"]

593

In [63]:
# 액션과 비슷한 벡터를 가진 단어 조회

word2vec.wv.most_similar("액션")

[('뒤', 0.998330295085907),
 ('향', 0.9982951283454895),
 ('계속', 0.9982795715332031),
 ('반', 0.9982011318206787),
 ('전부', 0.9981555342674255),
 ('순간', 0.9981482028961182),
 ('결심', 0.9981461763381958),
 ('만난', 0.9981397986412048),
 ('깊다', 0.9981038570404053),
 ('앞', 0.998100757598877)]

In [64]:
# 로맨스와 비슷한 벡터를 가진 단어 조회

word2vec.wv.most_similar("로맨스")

[('목숨', 0.9978252649307251),
 ('용기', 0.9976889491081238),
 ('도움', 0.9976135492324829),
 ('분', 0.997606635093689),
 ('소녀', 0.9975988268852234),
 ('공포', 0.9975889921188354),
 ('간다', 0.9975743889808655),
 ('매력', 0.9975569844245911),
 ('음', 0.997530996799469),
 ('소년', 0.9975164532661438)]

In [65]:
# 액션 Wordvector 조회
word2vec.wv["액션"]

array([ 0.08427072, -0.07716644,  0.05229799,  0.13992484, -0.05244315,
       -0.04213327, -0.00882047, -0.06753678, -0.13191444,  0.02196191,
       -0.06666358, -0.19645686, -0.08854065,  0.02158418,  0.15930362,
        0.17546763, -0.22305259, -0.05381698,  0.04790889, -0.20551835,
        0.05356871,  0.13317406,  0.09736171, -0.00537184,  0.00988167,
        0.0242842 ,  0.0465135 ,  0.2005962 , -0.21476531, -0.20322701,
        0.24572146, -0.05681239,  0.18275225, -0.07516666,  0.08023895,
        0.20067859,  0.2834339 , -0.06269107, -0.1636142 , -0.01832318,
        0.00446135,  0.11994823, -0.2276433 ,  0.01072078,  0.22947913,
        0.00676155, -0.14371853,  0.22909199,  0.10096107,  0.11460891,
        0.10370696,  0.06756728,  0.12095091, -0.09668256,  0.0822415 ,
        0.0482741 ,  0.12367057, -0.14206661, -0.08062828, -0.2189944 ,
       -0.0039569 , -0.04179502, -0.13769199,  0.01245143, -0.32849196,
       -0.17632177, -0.10005326,  0.22142625,  0.16752297,  0.25

In [66]:
# 로맨스 Wordvector 조회
word2vec.wv["로맨스"]

array([ 0.06093567, -0.06395797,  0.03026312,  0.1005477 , -0.03948276,
       -0.02042519, -0.01161245, -0.03807092, -0.09292899, -0.00106292,
       -0.05992204, -0.15213543, -0.0751387 ,  0.00627676,  0.11533029,
        0.13891663, -0.17637701, -0.03983949,  0.05833691, -0.17142263,
        0.04206053,  0.10413487,  0.08081177, -0.01161634,  0.00481065,
        0.02140458,  0.03760283,  0.15474403, -0.16340154, -0.16798645,
        0.18305582, -0.05363049,  0.13933751, -0.06733785,  0.06202348,
        0.16255811,  0.22187729, -0.03722227, -0.12421619, -0.00941596,
        0.00174527,  0.09529773, -0.17832513, -0.00077809,  0.19611987,
        0.00836941, -0.09630628,  0.16265754,  0.07180483,  0.07352922,
        0.06509214,  0.04957453,  0.09383751, -0.08064245,  0.06819927,
        0.02083845,  0.09394207, -0.12928893, -0.06363506, -0.15925978,
       -0.02078273, -0.04605674, -0.11761618,  0.01795474, -0.24762046,
       -0.12603806, -0.0932976 ,  0.17122225,  0.1391397 ,  0.20

In [67]:
word2vec_words = word2vec.wv.key_to_index.keys()
# word2vec_words

In [68]:
movie_df[["num","synopsis_clear_list"]][0:3]

,num,synopsis_clear_list
0,1,"[대체, 불가, 괴물, 형사, 마, 석도, 서울, 광수, 발탁, 베트남, 납치, 살..."
1,2,"[이, 근처, 폐허, 없다, 문, 있다, 규슈, 한적하다, 마을, 살, 있다, 소녀..."
2,3,"[불, 물, 공기, 흙, 개, 원소, 살, 있다, 엘리, 멘트, 시티, 재치, 있다..."


In [30]:
### 맛보기 3개 영화 줄거리만 wordvector 변환

for i in tqdm(range(3)):
    # 영화 제목
    title = movie_df.loc[i,"title"]
    # 전처리된 영화 줄거리
    line = movie_df.loc[i,"synopsis_clear_list"]
    
    doc2vec = None
    count = 0
    
    for word in line: # 영화 줄거리에서 한 단어씩 리턴
        # print("word = ", word)
        if word in word2vec_words: # 해당 단어가 Wordvector에 단어라면
            count += 1
            if doc2vec is None:
                doc2vec = word2vec.wv[word]
            else: # 첫번째 단어가 아니라면
                doc2vec = doc2vec + word2vec.wv[word] # 단어의 Wordvector 값 추가
                
    if doc2vec is not None:
        doc2vec = doc2vec / count # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나누어줌
        
    print(f"{title} is done!")

100%|██████████| 3/3 [00:00<00:00, 5698.78it/s]

범죄도시3 is done!
스즈메의 문단속 is done!
엘리멘탈 is done!


In [53]:
# 엘리멘탈 doc2vec 조회, 위의 마지막 doc2vec 이 엘리멘탈 이므로
# doc2vec

In [31]:
doc2vec.tostring()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18000\1786304619.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  doc2vec.tostring()


b'EEX=\x80tw\xbd\xeb\xc0\x8f=X`\xd2=\x1a\xa9\xf0\xbd\xcf\x8d\x8c\xbc;\x8f\xd9\xbb\xa8!\xaf\xbd\x891\x0e\xbe\t\xcd\xe4<Z\xc0\xe4\xbdSc<\xbe\n\x8a\x95\xbd\xd0\xf2<<\xe0\xa8\xde=\x88\x91\x0b>\xb5\x978\xbeQ=\xff\xbcK\x9c\x85\xbc\xf3L/\xbez\xf6\xc6<\xa1\xe9#>\x10\x80\x08>\x84\x07\xb2\xbc\x12\xd1\x0e= \xb0\x12=\xe2\xb7B=D\x9a#>]g\x1c\xbeO\xa2\x1d\xbeuC">\xcd\x13\xa7\xbd]\xf5!>\xcb\xe4\xfc\xbd\xa8\xa9\x02>\x17w.>R`H>\xbb\x11\xa7\xbcK\x88J\xbe\xc1^\xce\xbd\xfe\xf4i\xbd\xe4\x04\x03>`Od\xbe\x1a\xd1\x85=z\x0f_>\xeaT\xd4<\x94\x19%\xbe\xcc\x03>>\x88O\xe3<0\xba\x18=\xfb\x94\xbe=(\xb3<=\x07\xd7\xe8=u\xc1\x93\xbd\xa8d\x02>\xa8\xb5\x96=\xf5\x10\xbc=\\\xbd\x1c\xbe\xae:\xa2\xbc3\xd4Q\xbe~\xaa\xf7\xbc[\xc9\x87\xbd\x85 \xea\xbd\xda_\x8f=~\x91\x8b\xbe\xe5R3\xbe\xf8\xd1\x8d\xbd\x98\xc9f>9,9>\xbe\xa9s>\xd0sC=\x99Q\x11\xbe(\xdd\xce\xbd\x9b\xa3k=\xd4;J\xbd\xab\xcd\xd9=\xad|%=\xe9\xf0^\xbe6\xe5\xc0\xbd\xfe\xe9|\xbe'

In [32]:
numpy_array = np.fromstring(doc2vec.tostring(), dtype="float32")
numpy_array

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18000\2044994239.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  numpy_array = np.fromstring(doc2vec.tostring(), dtype="float32")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18000\2044994239.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  numpy_array = np.fromstring(doc2vec.tostring(), dtype="float32")


array([ 0.05280044, -0.06041384,  0.07019218,  0.10272282, -0.11751004,
       -0.01715746, -0.00663939, -0.08551341, -0.13886084,  0.0279298 ,
       -0.11169501, -0.18397264, -0.07301719,  0.0115325 ,  0.10872054,
        0.13629735, -0.18026622, -0.03115717, -0.01630988, -0.17119198,
        0.02428745,  0.16007091,  0.13330102, -0.0217321 ,  0.03486735,
        0.0358125 ,  0.04753865,  0.15976816, -0.15273805, -0.15393947,
        0.15846045, -0.08158074,  0.15816255, -0.12348326,  0.12760031,
        0.17037617,  0.19567993, -0.0203942 , -0.19778554, -0.10076667,
       -0.05711841,  0.12794834, -0.22295904,  0.06534024,  0.21783248,
        0.0259194 , -0.16123039,  0.18556136,  0.02774788,  0.03728694,
        0.0930576 ,  0.04606929,  0.11369138, -0.07214633,  0.1273371 ,
        0.07358867,  0.09182922, -0.1530661 , -0.01980337, -0.20491104,
       -0.03023266, -0.06630202, -0.11431984,  0.07000704, -0.2725944 ,
       -0.1751209 , -0.06924814,  0.2253784 ,  0.18083276,  0.23

# 전체 영화 전처리된 줄거리 Wordvector 값으로 진행 후 `DB` 저장

In [52]:
db = pymysql.connect(
    # host = "aws db 엔드포인트",
    host="",
    port=3306,
    user='admin',
    # passwd='관리자 계정 비밀번호',
    passwd="",
    db='movie_db',
    charset='utf8'
)

In [69]:
row_nums = len(movie_df)
row_nums

670

In [70]:
cursor = db.cursor()

In [71]:
cursor.execute("ALTER TABLE movie DROP COLUMN synopsis_vector")

0

In [72]:
db.commit()

In [73]:
cursor.execute("ALTER TABLE movie ADD synopsis_vector VARCHAR(500) DEFAULT NULL")

0

In [74]:
db.commit()

In [37]:
movie_df.loc[0,"num"]
movie_df.loc[0,"title"]
# movie_df.loc[0,"synopsis_clear_list"]

'범죄도시3'

In [75]:
for i in tqdm(range(row_nums)):
    # 영화 번호 조회
    num = movie_df.loc[i,"num"]
    # 영화 제목
    title = movie_df.loc[i,"title"]
    # 전처리된 영화 줄거리
    line = movie_df.loc[i,"synopsis_clear_list"]
    
    doc2vec = None
    count = 0
    for word in line: # 영화 줄거리에서 한 단어씩 리턴
        # print("word = ", word)
        if word in word2vec_words: # 해당 단어가 Wordvector에 단어라면
            count += 1
            if doc2vec is None:
                doc2vec = word2vec.wv[word]
            else: # 첫번째 단어가 아니라면
                doc2vec = doc2vec + word2vec.wv[word] # 단어의 Wordvector 값 추가
                
    if doc2vec is not None:
        doc2vec = doc2vec / count # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나누어줌
    
    # print(doc2vec.tostring())
    
    # movie 테이블에 영화 줄거리 wordvector 를 저장하는 쿼리
    try:
        sql = "update movie set synopsis_vector=%s where num=%s"
        
        # sql 쿼리 실행
        ## doc2vec.tostring() : 영화 줄거리 wordvector DB 저장
        cursor.execute(sql, (doc2vec.tostring(), num))
        db.commit()
    except Exception as e:
        print(e)
            
    # print(f"{title} is done!")

  0%|          | 0/670 [00:00<?, ?it/s]C:\Users\Playdata\AppData\Local\Temp\ipykernel_18000\1601703588.py:31: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  cursor.execute(sql, (doc2vec.tostring(), num))
100%|██████████| 670/670 [00:08<00:00, 74.61it/s]


### 🚀🚩

`doc2vec.tostring()` : 해당 데이터를 추가하려는 데이터베이스가 `MySQL`, 즉 `RDBMS` 이므로 `tostring()` 을 통해 
문자열로 변환을 해야만 데이터를 삽입할 수 있습니다.

#### 그러나

`객체 DB`, 통상적으로 많이 쓰이는 `Redis(레디스)` 와 같은 `NoSQL` DB를 쓴다면 `tostring()` 함수의 필요없이 그대로 데이터를 
DB에 저장이 가능합니다.

In [76]:
cursor.execute("select title, synopsis_vector from movie where num=1")

1

In [77]:
row_list = cursor.fetchall()

for i in row_list :
    print(i[0])
    print("="*100)
    
    print(i[1])
    print("="*100)

    print(np.fromstring(i[1],dtype="float32"))

범죄도시3
cv?=Y?????E=?U >?U??t???ֻ?????a	???%<?`??F?G??[????<PW*>?=>??s???{??^=?a???4=?[>f:?=??B?36x<u??<??~= ?N>??X?Ckc???y>Mm??].5>L??zz?=?-T>@H?>??x??$/??rؼ??9b??=??f?AB<?~>e]??p?L;h>??=???=?I?=?=S?>y?ӽwO?=??:=>????u???f??6???`??r?c?<?2??>?3???轙?a>??;>j?>#	=?b	?ힽ`?=??輦??=??2=R??t_?Y|?
[ 4.6743762e-02  7.4706036e-01  4.8156019e-02  1.2532519e-01
  7.4739450e-01  5.5565572e-01 -6.5382863e-03  7.4705881e-01
  5.3664011e-01  1.0085880e-02  7.4756235e-01  7.7830923e-01
  7.4748605e-01  1.1672750e-02  1.6634870e-01  1.8481106e-01
  9.5018381e-01  9.8143381e-01  5.4259308e-02  8.7933725e-01
  4.4005629e-02  1.3901995e-01  4.6686552e-02  7.5877756e-01
  1.5149641e-02  1.1672844e-02  6.2072035e-02  2.0141220e-01
  8.4471506e-01  8.8835543e-01  2.4340533e-01  7.4776155e-01
  1.7693467e-01  7.4643397e-01  4.6747662e-02  2.0720385e-01
  1.8679905e-01  9.6971506e-01  6.8414682e-01 -2.6421664e-02
  1.8219369e-04  4.6691306e-02  8.9940256e-01  1.1844457e-02
  2.4816607e-01

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18000\3409460628.py:10: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  print(np.fromstring(i[1],dtype="float32"))


In [78]:
db.close()